In [47]:
import os

import pandas as pd

ROOT_DIR = '/home/ddon0001/PhD/data/cell_tracking_challenge/ST/'
NO_DIV_ROOT = '/home/ddon0001/PhD/experiments/scaled/no_div_constraint/'
in_csv_path = os.path.join(NO_DIV_ROOT, 'summary.csv')
datasets = pd.read_csv(in_csv_path)

In [48]:
# are there any edges with fractional flow?
ds_names = []
no_div_frac = []
for _, row in datasets.iterrows():
    no_div_sol_path = os.path.join(NO_DIV_ROOT, row['ds_name'], 'all_edges.csv')
    no_div_edges = pd.read_csv(no_div_sol_path)
    count_no_div_frac = len(no_div_edges[no_div_edges['flow'] % 1 != 0])
    ds_names.append(row['ds_name'])
    no_div_frac.append(count_no_div_frac)
summary_df = pd.DataFrame({'ds_name': ds_names, 'no_div_frac': no_div_frac})

In [49]:
summary_df

,ds_name,no_div_frac
0,Fluo-C3DL-MDA231_01,0
1,Fluo-C3DL-MDA231_02,0
2,Fluo-N2DH-GOWT1_01,0
3,Fluo-N2DH-GOWT1_02,0
4,PhC-C2DH-U373_01,0
5,PhC-C2DH-U373_02,0
6,Fluo-N2DL-HeLa_01,0
7,Fluo-N2DL-HeLa_02,0
8,Fluo-C2DL-MSC_01,0
9,Fluo-C2DL-MSC_02,0


In [60]:
# what's happened with our merges and with our divisions
import networkx as nx

ds_names = []
no_div_merge = []
no_div_div = []
gt_div = []
for _, row in datasets.iterrows():
    no_div_sol_path = os.path.join(NO_DIV_ROOT, row['ds_name'], 'matched_solution.graphml')
    gt_div_sol_path = os.path.join(NO_DIV_ROOT, row['ds_name'], 'matched_gt.graphml')

    no_div_sol = nx.read_graphml(no_div_sol_path, node_type=int)
    gt_sol = nx.read_graphml(gt_div_sol_path)

    count_no_div_merges = len([node for node in no_div_sol.nodes if no_div_sol.in_degree(node) > 1])
    
    count_no_div_div = len([node for node in no_div_sol.nodes if no_div_sol.out_degree(node) > 1])
    count_gt_div = len([node for node in gt_sol.nodes if gt_sol.out_degree(node) > 1])

    ds_names.append(row['ds_name'])
    no_div_merge.append(count_no_div_merges)
    no_div_div.append(count_no_div_div)
    gt_div.append(count_gt_div)

merge_df = pd.DataFrame({
    'ds_name': ds_names,
    'merges': no_div_merge,
    'divs': no_div_div,
    'gt_div': gt_div
    })

print(merge_df)

                ds_name  merges  divs  gt_div
0   Fluo-C3DL-MDA231_01       1     0       0
1   Fluo-C3DL-MDA231_02       4     0       1
2    Fluo-N2DH-GOWT1_01       3     3       2
3    Fluo-N2DH-GOWT1_02      10    13       4
4      PhC-C2DH-U373_01       0     0       0
5      PhC-C2DH-U373_02       0     0       0
6     Fluo-N2DL-HeLa_01      15    93      94
7     Fluo-N2DL-HeLa_02      32   220     209
8      Fluo-C2DL-MSC_01       0     0       0
9      Fluo-C2DL-MSC_02       0     0       0
10    Fluo-C3DH-H157_01       0     0       0
11    Fluo-C3DH-H157_02       0     0       0
12     DIC-C2DH-HeLa_01       0     1       8
13     DIC-C2DH-HeLa_02       0     1       5
14     Fluo-N3DH-CHO_01       0     0       4
15     Fluo-N3DH-CHO_02       0     0       3
16      BF-C2DL-MuSC_01      34    56      22
17      BF-C2DL-MuSC_02      49    69      22
18       BF-C2DL-HSC_01       1    12      11
19       BF-C2DL-HSC_02       4   160     157
20    Fluo-C3DH-A549_01       0   

In [ ]:
# What previously fueled our divisions (cheating, "proper division", or prior merges)
from collections import defaultdict

ds_names = []
div_cheating = []
div_proper = []
div_immediate_merge = []
div_prior_merge = []
div_unresolved = []
div_parent_of_merge = []
unresolved = defaultdict(list)
for _, row in datasets.iterrows():
    div_sol_path = os.path.join(DIV_ROOT, row['ds_name'], 'matched_solution.graphml')
    div_sol_all_edges_path = os.path.join(DIV_ROOT, row['ds_name'], 'all_edges.csv')

    div_sol = nx.read_graphml(div_sol_path, node_type=int)
    all_edges = pd.read_csv(div_sol_all_edges_path)
    parent_nodes = [node for node in div_sol.nodes if div_sol.out_degree(node) > 1]
    count_cheating = 0
    count_proper = 0
    count_immediate_merge = 0
    count_prior_merge = 0
    count_parent_of_merge = 0
    for parent in parent_nodes:
        children = list(div_sol.successors(parent))
        # flow from div is entering children so this is a cheat
        if any([len(flow := (all_edges[(all_edges.u == -3) & (all_edges.v == child)]['flow'].values)) == 1 and flow[0] == 0.5 for child in children]):
            count_cheating += 1
            continue
        # 1 div flow into parent means it's a "proper" division
        if len(flow := all_edges[(all_edges.u == -3) & (all_edges.v == parent)]['flow'].values) and flow[0] == 1:
            count_proper += 1
            continue
        # if both children have two real parents
        if all([len(list(div_sol.predecessors(child))) > 1 for child in children]):
            count_parent_of_merge += 1
            continue
        # sum flow from all real predecessors into parent is greater than 1, so it's a merge
        predecessors = list(div_sol.predecessors(parent))
        pred_flow = sum([div_sol.edges[predecessor, parent]['flow'] for predecessor in predecessors])
        if pred_flow > 1:
            # more than one predecessor for parent means we just merged
            if len(predecessors) > 1:
                count_immediate_merge += 1
            # just one predecessor for parent means we merged in prior frames
            else:
                count_prior_merge += 1
            continue
        unresolved[row['ds_name']].append(parent)
    ds_names.append(row['ds_name'])
    div_cheating.append(count_cheating)
    div_proper.append(count_proper)
    div_immediate_merge.append(count_immediate_merge)
    div_prior_merge.append(count_prior_merge)
    div_parent_of_merge.append(count_parent_of_merge)
    div_unresolved.append(len(unresolved[row['ds_name']]))
old_div_df = pd.DataFrame({
    'ds_name': ds_names,
    'old_cheating': div_cheating,
    'old_proper': div_proper,
    'old_immediate_merge': div_immediate_merge,
    'old_prior_merge': div_prior_merge,
    'old_parent_of_merge': div_parent_of_merge,
    'old_unresolved': div_unresolved,
})
print(old_div_df)

In [50]:
from utils import classify_all_divisions

new_div_df, all_divs = classify_all_divisions(NO_DIV_ROOT, write_out=True)

In [51]:
new_div_df

,ds_name,new_proper,new_immediate_merge,new_cheating,new_prior_merge,new_super_merge,new_super_parent,total
0,Fluo-C3DL-MDA231_01,0,0,0,0,0,0,0
1,Fluo-C3DL-MDA231_02,0,0,0,0,0,0,0
2,Fluo-N2DH-GOWT1_01,1,1,0,1,0,0,3
3,Fluo-N2DH-GOWT1_02,5,1,1,6,0,0,13
4,PhC-C2DH-U373_01,0,0,0,0,0,0,0
5,PhC-C2DH-U373_02,0,0,0,0,0,0,0
6,Fluo-N2DL-HeLa_01,58,5,20,9,0,1,93
7,Fluo-N2DL-HeLa_02,183,8,16,12,0,1,220
8,Fluo-C2DL-MSC_01,0,0,0,0,0,0,0
9,Fluo-C2DL-MSC_02,0,0,0,0,0,0,0


In [52]:
new_div_df[new_div_df.total != 0]

,ds_name,new_proper,new_immediate_merge,new_cheating,new_prior_merge,new_super_merge,new_super_parent,total
2,Fluo-N2DH-GOWT1_01,1,1,0,1,0,0,3
3,Fluo-N2DH-GOWT1_02,5,1,1,6,0,0,13
6,Fluo-N2DL-HeLa_01,58,5,20,9,0,1,93
7,Fluo-N2DL-HeLa_02,183,8,16,12,0,1,220
12,DIC-C2DH-HeLa_01,1,0,0,0,0,0,1
13,DIC-C2DH-HeLa_02,1,0,0,0,0,0,1
16,BF-C2DL-MuSC_01,21,19,0,14,2,2,56
17,BF-C2DL-MuSC_02,22,38,0,7,0,2,69
18,BF-C2DL-HSC_01,11,1,0,0,0,0,12
19,BF-C2DL-HSC_02,133,1,23,2,0,1,160


In [55]:
# wb penalty divisions?
PENALTY_ROOT = '/home/ddon0001/PhD/experiments/flow_penalty/st/'
penalty_div_df, penalty_divs = classify_all_divisions(PENALTY_ROOT, write_out=True)

In [56]:
# wb chonky penalty divisions
PENALTY_ROOT_50 = '/home/ddon0001/PhD/experiments/flow_penalty/st_50/'
penalty_div_50_df, penalty_divs = classify_all_divisions(PENALTY_ROOT_50, write_out=True)

In [53]:
# how many edges in each solution are superunitary?
import networkx as nx
ds_names = []
superunitary_flow = []
for _, row in datasets.iterrows():
    div_sol_path = os.path.join(NO_DIV_ROOT, row['ds_name'], 'matched_solution.graphml')
    div_sol_all_edges_path = os.path.join(NO_DIV_ROOT, row['ds_name'], 'all_edges.csv')

    count_super = 0
    div_sol = nx.read_graphml(div_sol_path, node_type=int)
    for edge in div_sol.edges:
        if div_sol.edges[edge]['flow'] > 1:
            count_super += 1
    ds_names.append(row['ds_name'])
    superunitary_flow.append(count_super)
super_df = pd.DataFrame({
    'ds_name': ds_names,
    'num_super_edges': superunitary_flow
})
super_df.sort_values('num_super_edges')

,ds_name,num_super_edges
0,Fluo-C3DL-MDA231_01,0
4,PhC-C2DH-U373_01,0
5,PhC-C2DH-U373_02,0
12,DIC-C2DH-HeLa_01,0
11,Fluo-C3DH-H157_02,0
10,Fluo-C3DH-H157_01,0
9,Fluo-C2DL-MSC_02,0
8,Fluo-C2DL-MSC_01,0
14,Fluo-N3DH-CHO_01,0
13,DIC-C2DH-HeLa_02,0


In [54]:
# how many super unitary edges left after penalty=50?
import os
import networkx as nx

PENALTY_ROOT = '/home/ddon0001/PhD/experiments/flow_penalty/st_50/'
PENALTY_DS = os.path.join(PENALTY_ROOT, 'summary.csv')
penalty_super = pd.read_csv(PENALTY_DS)

ds_names = []
superunitary_flow = []
for _, row in penalty_super.iterrows():
    div_sol_path = os.path.join(PENALTY_ROOT, row['ds_name'], 'matched_solution.graphml')
    div_sol_all_edges_path = os.path.join(PENALTY_ROOT, row['ds_name'], 'all_edges.csv')

    count_super = 0
    div_sol = nx.read_graphml(div_sol_path, node_type=int)
    for edge in div_sol.edges:
        if div_sol.edges[edge]['flow'] > 1:
            count_super += 1
    ds_names.append(row['ds_name'])
    superunitary_flow.append(count_super)
penalty_super_df = pd.DataFrame({
    'ds_name': ds_names,
    'num_super_edges': superunitary_flow
})
penalty_super_df.sort_values('num_super_edges')

,ds_name,num_super_edges
0,Fluo-C3DL-MDA231_01,0
1,Fluo-C3DL-MDA231_02,0
2,Fluo-N2DH-GOWT1_01,0
4,PhC-C2DH-U373_01,0
6,Fluo-N2DL-HeLa_01,0
5,PhC-C2DH-U373_02,0
9,Fluo-C2DL-MSC_02,0
8,Fluo-C2DL-MSC_01,0
12,DIC-C2DH-HeLa_01,0
13,DIC-C2DH-HeLa_02,0


In [57]:
new_div_df[new_div_df.total != 0]

,ds_name,new_proper,new_immediate_merge,new_cheating,new_prior_merge,new_super_merge,new_super_parent,total
2,Fluo-N2DH-GOWT1_01,1,1,0,1,0,0,3
3,Fluo-N2DH-GOWT1_02,5,1,1,6,0,0,13
6,Fluo-N2DL-HeLa_01,58,5,20,9,0,1,93
7,Fluo-N2DL-HeLa_02,183,8,16,12,0,1,220
12,DIC-C2DH-HeLa_01,1,0,0,0,0,0,1
13,DIC-C2DH-HeLa_02,1,0,0,0,0,0,1
16,BF-C2DL-MuSC_01,21,19,0,14,2,2,56
17,BF-C2DL-MuSC_02,22,38,0,7,0,2,69
18,BF-C2DL-HSC_01,11,1,0,0,0,0,12
19,BF-C2DL-HSC_02,133,1,23,2,0,1,160


In [59]:
penalty_div_50_df[penalty_div_50_df.total != 0]

,ds_name,new_proper,new_immediate_merge,new_cheating,new_prior_merge,new_super_merge,new_super_parent,total
2,Fluo-N2DH-GOWT1_01,1,1,0,0,0,0,2
3,Fluo-N2DH-GOWT1_02,8,4,0,3,0,0,15
6,Fluo-N2DL-HeLa_01,79,24,0,0,0,1,104
7,Fluo-N2DL-HeLa_02,198,37,0,2,0,1,238
12,DIC-C2DH-HeLa_01,1,0,0,0,0,0,1
13,DIC-C2DH-HeLa_02,1,0,0,0,0,0,1
16,BF-C2DL-MuSC_01,25,55,0,0,0,1,81
17,BF-C2DL-MuSC_02,23,48,0,0,0,2,73
18,BF-C2DL-HSC_01,11,1,0,0,0,0,12
19,BF-C2DL-HSC_02,156,4,0,0,0,1,161


In [58]:
penalty_div_df[penalty_div_df.total != 0]

,ds_name,new_proper,new_immediate_merge,new_cheating,new_prior_merge,new_super_merge,new_super_parent,total
2,Fluo-N2DH-GOWT1_01,1,1,0,1,0,0,3
3,Fluo-N2DH-GOWT1_02,5,1,1,6,0,0,13
6,Fluo-N2DL-HeLa_01,71,8,7,7,0,1,94
7,Fluo-N2DL-HeLa_02,191,11,7,11,0,1,221
12,DIC-C2DH-HeLa_01,1,0,0,0,0,0,1
13,DIC-C2DH-HeLa_02,1,0,0,0,0,0,1
16,BF-C2DL-MuSC_01,20,34,0,12,2,2,68
17,BF-C2DL-MuSC_02,22,47,0,2,0,2,73
18,BF-C2DL-HSC_01,11,1,0,0,0,0,12
19,BF-C2DL-HSC_02,149,2,7,1,0,1,160


In [1]:
# how does GOWT1_02 improve in best solution (penalty=12)
import os
import networkx as nx
import pandas as pd

old_sol_path = '/home/ddon0001/PhD/experiments/scaled/no_div_constraint/Fluo-N2DH-GOWT1_02/matched_solution.graphml'
new_sol_path = '/home/ddon0001/PhD/experiments/grid_again/best/Fluo-N2DH-GOWT1_02_12.0/matched_solution.graphml'

old_sol = nx.read_graphml(old_sol_path, node_type=int)
new_sol = nx.read_graphml(new_sol_path, node_type=int)


In [2]:
new_sol.edges[0, 22]

{'flow': 1.0,
 'cost': 0.9772048530677026,
 'EdgeFlag.FALSE_POS': False,
 'EdgeFlag.WRONG_SEMANTIC': False,
 'EdgeFlag.INTERTRACK_EDGE': False}

In [3]:
old_ws = set([edge for edge in old_sol.edges if old_sol.edges[edge]['EdgeFlag.WRONG_SEMANTIC']])
old_fp = set([edge for edge in old_sol.edges if old_sol.edges[edge]['EdgeFlag.FALSE_POS']])

new_ws = set([edge for edge in new_sol.edges if new_sol.edges[edge]['EdgeFlag.WRONG_SEMANTIC']])
new_fp = set([edge for edge in new_sol.edges if new_sol.edges[edge]['EdgeFlag.FALSE_POS']])

ws_diff = old_ws - new_ws
fp_diff = old_fp - new_fp

were_ws = old_ws - new_ws
became_ws = new_ws - old_ws
were_fp = old_fp - new_fp
became_fp = new_fp - old_fp
print('Were ws: ', were_ws)
print('Became ws: ', became_ws)
print()
print('Were fp: ', were_fp)
print('Became fp: ', became_fp)

Were ws:  {(1460, 1489), (2175, 2202), (540, 563), (849, 872), (736, 758), (2060, 2085), (2049, 2075), (863, 886), (841, 863)}
Became ws:  {(2047, 2073), (882, 905), (747, 769), (2073, 2100)}

Were fp:  {(1467, 1489), (538, 563), (2059, 2092), (2177, 2202), (863, 906), (849, 871), (725, 758), (2045, 2075), (839, 863), (2060, 2093)}
Became fp:  {(725, 769), (839, 871), (2073, 2092), (2045, 2073), (882, 906), (2059, 2093)}


In [4]:
ws_nodes_of_interest = set([item for pair in were_ws for item in pair]).union(set([item for pair in became_ws for item in pair]))
fp_nodes_of_interest = set([item for pair in were_fp for item in pair]).union(set([item for pair in became_fp for item in pair]))


ws_subgraph_old = old_sol.subgraph(ws_nodes_of_interest)
ws_subgraph_new = new_sol.subgraph(ws_nodes_of_interest)

fp_subgraph_old = old_sol.subgraph(fp_nodes_of_interest)
fp_subgraph_new = new_sol.subgraph(fp_nodes_of_interest)

In [5]:
# we need layers for full old sol, full new sol, the images, the seg and the ws/fp edges
import napari
import networkx as nx
from tracktour import load_tiff_frames
from napari.layers import Graph
from napari_graph import DirectedGraph

def nx_graph_to_napari(nxg, location_keys, frame_key):
    pos_keys = [frame_key] + list(location_keys)
    pos = {
        node: tuple([nxg.nodes[node][k] for k in pos_keys])
        for node in nxg.nodes
    }
    nx.set_node_attributes(nxg, pos, "pos")
    napari_graph = DirectedGraph.from_networkx(nxg)
    graph_layer = Graph(
        napari_graph,
        out_of_slice_display=True,
        ndim=3,
        size=5,
    )
    return graph_layer


In [6]:
data_root = '/home/ddon0001/PhD/data/cell_tracking_challenge/ST/Fluo-N2DH-GOWT1/'
ims_path = os.path.join(data_root, '02')
seg_path = os.path.join(data_root, '02_ST/SEG/')

old_sol_full = nx_graph_to_napari(old_sol, ['y', 'x'], 't')
old_sol_full.name = 'old_sol'
new_sol_full = nx_graph_to_napari(new_sol, ['y', 'x'], 't')
new_sol_full.name = 'new_sol'

# ws_old = nx_graph_to_napari(ws_subgraph_old, ['y', 'x'], 't')
# ws_old.name = 'ws_old'
# ws_new = nx_graph_to_napari(ws_subgraph_new, ['y', 'x'], 't')
# ws_new.name = 'ws_new'

# fp_old = nx_graph_to_napari(fp_subgraph_old, ['y', 'x'], 't')
# fp_old.name = 'fp_old'
# fp_new = nx_graph_to_napari(fp_subgraph_new, ['y', 'x'], 't')
# fp_new.name = 'fp_new'


In [7]:
viewer = napari.Viewer()

ims = viewer.add_image(load_tiff_frames(ims_path), name='ims')
seg = viewer.add_labels(load_tiff_frames(seg_path), name='seg')

viewer.add_layer(old_sol_full)
viewer.add_layer(new_sol_full)

Invalid schema for package 'tracktour', please run 'npe2 validate tracktour' to check for manifest errors.


<Graph layer 'new_sol' at 0x7f3fa8bfb8b0>

In [8]:
were_ws

{(540, 563),
 (736, 758),
 (841, 863),
 (849, 872),
 (863, 886),
 (1460, 1489),
 (2049, 2075),
 (2060, 2085),
 (2175, 2202)}

In [44]:
became_ws

{(747, 769), (882, 905), (2047, 2073), (2073, 2100)}

In [45]:
became_fp

{(725, 769), (839, 871), (882, 906), (2045, 2073), (2059, 2093), (2073, 2092)}

In [43]:
old_sol.nodes[2075]

{'t': 83,
 'y': 734.0771812080537,
 'x': 458.41442953020135,
 'label': 37,
 'enter_exit_cost': 69.58147651006709,
 'div_cost': 36.65332118775408,
 'NodeFlag.TRUE_POS': True,
 'NodeFlag.NON_SPLIT': False,
 'NodeFlag.FALSE_POS': False,
 'pos': (83, 734.0771812080537, 458.41442953020135)}

In [40]:
list(new_sol.predecessors(1868))

[1841]

In [61]:
gt_path = '/home/ddon0001/PhD/experiments/scaled/no_div_constraint/Fluo-N2DH-GOWT1_02/matched_gt.graphml'
gt_sol = nx.read_graphml(gt_path)
gt_layer = nx_graph_to_napari(gt_sol, ['x', 'y'], 't')
gt_layer.name = 'gt'
viewer.add_layer(gt_layer)

/home/ddon0001/PhD/code/napari-graph/src/napari_graph/base_graph.py:858: UserWarning: Node IDs must be integers. They have been converted automatically.
  warnings.warn(


<Graph layer 'gt' at 0x7f3f90293130>

In [ ]:
list(gt_sol.successors('49_19'))

In [ ]:
old_sol.nodes[425]

In [ ]:
gt_sol.nodes['49_19']

In [42]:
big_penalty = '/home/ddon0001/PhD/experiments/flow_penalty/Fluo-N2DH-GOWT1_02_300/matched_solution.graphml'
big_penalty_sol = nx.read_graphml(big_penalty, node_type=int)
superunitary_flow = [edge for edge in big_penalty_sol.edges if big_penalty_sol.edges[edge]['flow'] > 1]
print(superunitary_flow)

[]
